# C 프로그램 전체 진행과정 살펴보기
## 1. 단일 파일 실행 예(hello.c) : 소스 작성 => 컴파일(어셈블리어 > 기계어) => 링킹 => 실행가능한 파일

In [1]:
%pwd
#현재 위치 확인 % >> 라인명령어

'/home/jovyan/work'

In [2]:
#1. 프로젝트 생성(폴더 생성) : project1  ! >> 수행명령어
!mkdir project1

In [3]:
%%writefile project1/hello.c
//2. 프로젝트내에 C 파일 생성: project1/hello.c 
#include <stdio.h> 헤더파일 >> 함수 덩어리 파일
int main(){
    printf("Hello, world !!!");
    return 0;
}

Writing project1/hello.c


In [5]:
#3. 프로젝트내에  C 파일을 어셈블리어로 변환( hello.c --> hello.s ) 
#명령 옵션: -S
!gcc -S project1/hello.c -o project1/hello.s

project1/hello.c:2:20: warning: extra tokens at end of #include directive
    2 | #include <stdio.h> 헤더파일 >> 함수 덩어리 파일
      |                    ^~~~~~~~


In [6]:
#4. 프로젝트내에 어셈블리어 내용 확인(hello.s)
!cat project1/hello.s

	.file	"hello.c"
	.text
	.section	.rodata
.LC0:
	.string	"Hello, world !!!"
	.text
	.globl	main
	.type	main, @function
main:
.LFB0:
	.cfi_startproc
	endbr64
	pushq	%rbp
	.cfi_def_cfa_offset 16
	.cfi_offset 6, -16
	movq	%rsp, %rbp
	.cfi_def_cfa_register 6
	leaq	.LC0(%rip), %rax
	movq	%rax, %rdi
	movl	$0, %eax
	call	printf@PLT
	movl	$0, %eax
	popq	%rbp
	.cfi_def_cfa 7, 8
	ret
	.cfi_endproc
.LFE0:
	.size	main, .-main
	.ident	"GCC: (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0"
	.section	.note.GNU-stack,"",@progbits
	.section	.note.gnu.property,"a"
	.align 8
	.long	1f - 0f
	.long	4f - 1f
	.long	5
0:
	.string	"GNU"
1:
	.align 8
	.long	0xc0000002
	.long	3f - 2f
2:
	.long	0x3
3:
	.align 8
4:


In [7]:
#5. 어셈블리어 파일 -> 목적 파일(Object File)로 변환: C,C++ 등의 고수준 언어로 작성된 코드가 기계어로 변환된 중간 파일로 곧 최종 실행 파일을 
#   만들기 위해 **링커(Linker)**를 통해 다른 목적 파일과 결합해야 함.
# 명령 옵션: -c
!gcc -c project1/hello.s -o project1/hello.o

In [8]:
#6. 목적 파일 -> 실행가능한 파일로 변환
!gcc project1/hello.o -o project1/hello.exe

In [9]:
#7. 실행
!./project1/hello.exe

Hello, world !!!

## 2. 다중 파일 컴파일 예시  
📂 **프로젝트 구성**

```
project2/
│── main.c  # 메인 프로그램
│── utils.c # 유틸리티 함수 
│── utils.h # 유틸리티 함수 헤더 파일
```

#### ✅ **방법 1: 목적 파일 없이 직접 컴파일 (비효율적)**  

```sh
gcc project2/main.c project2/utils.c -o project2/my_program.exe
```

- `main.c`, `utils.c` 모두 매번 다시 컴파일됨 → **비효율적**  
- 수정하지 않은 `utils.c`도 계속 컴파일해야 함.

#### ✅ **방법 2: 목적 파일을 활용하여 컴파일 (효율적)**  

```sh
gcc -c project2/main.c -o project2/main.o
gcc -c project2/utils.c -o project2/utils.o
gcc project2/main.o project2/utils.o -o project2/my_program.exe
```

- `utils.c`가 수정되지 않으면 **다시 컴파일할 필요 없음**  
- 변경된 파일만 재컴파일하면 됨 → **빌드 속도 향상**


In [ ]:
%pwd 
#현재 위치 확인

In [10]:
#1. 프로젝트 폴더 생성
!mkdir project2

In [11]:
%%writefile project2/utils.h
//2-1. C프로그램 utils.h 파일 생성
#ifndef UTILS_H
#define UTILS_H
void myprintf();
#endif //UTILS_H 중복 방지를 위해

Writing project2/utils.h


In [12]:
%%writefile project2/utils.c
//2-2. C프로그램 utils.c 파일
#include <stdio.h>
#include "utils.h"
void myprintf(){
    printf("내가 만들 유틸리티 함수: myprintf()\n");    
}

Writing project2/utils.c


In [13]:
%%writefile project2/main.c
//2-3. C프로그램 main.c 파일
#include <stdio.h>
#include "utils.h"
int main(){
    printf("This is main.c\n");
    myprintf();
    return 0;
}

Writing project2/main.c


In [14]:
#3. 개별 C파일을 목적 파일(.o)로 변환
!gcc -c project2/utils.c -o project2/utils.o
!gcc -c project2/main.c -o project2/main.o

In [15]:
#4. 목적 파일을 실행가능한 파일로 링크
!gcc project2/main.o project2/utils.o -o project2/main.exe

In [16]:
#5. 결과 확인
!./project2/main.exe

This is main.c
내가 만들 유틸리티 함수: myprintf()


이렇게 실행파일이 만들어지는구나.. 
(소스파일 > 어셈블리 > 오브젝트 (목적) > 링커 > exe 실행파일